In [ ]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report
import itertools

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, BatchNormalization, Activation, Dense, Flatten, Dropout, LeakyReLU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics  import binary_crossentropy, categorical_crossentropy, mean_squared_error
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.keras.models import model_from_json
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import efficientnet as efn

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num of GPUs Availavle: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
ROOT_DIR = os.path.abspath('./')
DATA_PATH = os.path.join('path', "Beat_Images_(234x234)")
DATA_PATH

In [ ]:
train_path = os.path.join(DATA_PATH, 'Train')
train_path

In [ ]:
test_path = os.path.join(DATA_PATH, 'Test')
test_path

In [ ]:
train_datagen = ImageDataGenerator(validation_split=0.1, rescale=1. / 255)

In [ ]:
train_batches = train_datagen.flow_from_directory(directory=train_path, 
                                            target_size=(234,234), 
                                            classes=['NOR', 'APC', 'FPNB', 'FVNB', 'LBB', 'PAB', 'PVC', 'RBB'], 
                                            batch_size=128,
                                            subset='training')


In [ ]:
train_batches.class_indices

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_batches.classes), 
            train_batches.classes)

In [ ]:
class_weights

In [ ]:
class_weights = {
    0: 0.17244932,
    1: 4.36603881,
    2: 33.5495614,
    3: 13.52422207,
    4: 1.90356858,
    5: 2.97684465,
    6: 2.01636968,
    7: 1.95614259
}


In [ ]:
validation_batches = train_datagen.flow_from_directory(directory=train_path, 
                                            target_size=(234,234), 
                                            classes=['NOR', 'APC', 'FPNB', 'FVNB', 'LBB', 'PAB', 'PVC', 'RBB'], 
                                            batch_size=128,
                                            subset='validation')

In [ ]:
validation_batches.class_indices

In [ ]:
test_batches = ImageDataGenerator(rescale=1. / 255).flow_from_directory(directory=test_path, 
                                                                        target_size=(234,234), 
                                                                        classes=['NOR', 'APC', 'FPNB', 'FVNB', 'LBB', 'PAB', 'PVC', 'RBB'], 
                                                                        batch_size=128)

In [ ]:
test_batches.class_indices

In [ ]:
# imgs, labels = next(train_batches)

In [ ]:
# def plotImages(images_arr):
#     fig, axes = plt.subplots(1, 24, figsize=(20,20))
#     axes = axes.flatten()
#     for img, ax in zip( images_arr, axes):
#         ax.imshow(img)
#         ax.axis('off')
#     plt.tight_layout()
#     plt.show()

In [ ]:
# plotImages(imgs)
# print(labels)

In [ ]:
len(train_batches)

In [ ]:
# # Custom Function to calculate F1 Score 

# def f1(y_true, y_pred):
#     def recall(y_true, y_pred):
#         """Recall metric.

#         Only computes a batch-wise average of recall.

#         Computes the recall, a metric for multi-label classification of
#         how many relevant items are selected.
#         """
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#         recall = true_positives / (possible_positives + K.epsilon())
#         return recall

#     def precision(y_true, y_pred):
#         """Precision metric.

#         Only computes a batch-wise average of precision.

#         Computes the precision, a metric for multi-label classification of
#         how many selected items are relevant.
#         """
#         true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#         predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#         precision = true_positives / (predicted_positives + K.epsilon())
#         return precision
    
#     precision = precision(y_true, y_pred)
#     recall = recall(y_true, y_pred)
    
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
checkpoint_filepath = 'path\\best.h5'

In [ ]:
callbacks = ModelCheckpoint(filepath= checkpoint_filepath,  monitor="val_accuracy", verbose=1, save_best_only=True, 
                            save_weights_only=True, mode="max")

# Networks

# Custom Model_1

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(234, 234, 3)))
# model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.5))


model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))
model.add(Dense(8, activation='softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adadelta', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=100, class_weight=class_weights, callbacks=callbacks)

In [ ]:
# Loss Curve

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy Curve

plt.plot(history.history['accuracy'], label='Train accuracy')
plt.plot(history.history['val_accuracy'], label='Val accuracy')
plt.xlabel('Ephocs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
#Model Evalution
score, acc = model.evaluate(x=test_batches, steps=len(test_batches))

print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
predictions = model.predict(x=test_batches, steps=len(test_batches))
predictions

In [ ]:
predictions = np.argmax(predictions, axis=-1)
predictions

In [ ]:
print("Classification Report: \n", classification_report(test_batches.classes, predictions))

In [ ]:
#Saving the model

fer_json = model.to_json()
with open("path\\model_234x234_nvs_BN.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("path\\model_234x234_nvs_BN.h5")

# Custom Model_2

In [ ]:
model_2 = Sequential()

model_2.add(Conv2D(16, (3, 3), padding='same', activation = 'relu', input_shape=(234, 234, 3)))
model_2.add(Conv2D(32, (3, 3), padding='same', activation = 'relu'))
# model_2.add(BatchNormalization())
model_2.add(MaxPool2D(pool_size=(2, 2), strides= 2))
model_2.add(Dropout(0.5))

model_2.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
model_2.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
# model.add(BatchNormalization())
model_2.add(MaxPool2D(pool_size=(2, 2), strides= 2))
model_2.add(Dropout(0.5))

model_2.add(Flatten())

model_2.add(Dense(144, activation='relu'))
model_2.add(Dense(8, activation='softmax'))



In [ ]:
model_2.summary()

In [ ]:
model_2.compile(optimizer = 'sgd', loss = 'mean_squared_error', metrics = ['accuracy'])

In [ ]:
history = model_2.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=100, class_weight=class_weights, callbacks=callbacks)


In [ ]:
# Loss Curve

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy Curve

plt.plot(history.history['accuracy'], label='Train accuracy')
plt.plot(history.history['val_accuracy'], label='Val accuracy')
plt.xlabel('Ephocs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
#Model Evalution
score, acc = model_2.evaluate(x=test_batches, steps=len(test_batches))

print('Test score:', score)
print('Test accuracy:', acc)
# print('Test F1 Score:', f1_score)

In [ ]:
predictions = model_2.predict(x=test_batches, steps=len(test_batches))
predictions

In [ ]:
predictions = np.argmax(predictions, axis=-1)
predictions

In [ ]:
print("Classification Report: \n", classification_report(test_batches.classes, predictions))

In [ ]:
#Saving the model

fer_json = model_2.to_json()
with open("path\\model_2.json", "w") as json_file:
    json_file.write(fer_json)
model_2.save_weights("path\\model_2.h5")

# Custom Model_3

In [ ]:
model_3 = Sequential()
model_3.add(Conv2D(16, (5, 5), padding='same', activation = 'relu', input_shape=(234, 234, 3)))
model_3.add(Conv2D(32, (5, 5), padding='same', activation = 'relu'))
# model_3.add(BatchNormalization())
model_3.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model_3.add(Dropout(0.5))

model_3.add(Conv2D(64, (5, 5), padding='same', activation = 'relu'))
model_3.add(Conv2D(64, (5, 5), padding='same', activation = 'relu'))
# model_3.add(BatchNormalization())
model_3.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model_3.add(Dropout(0.5))

model_3.add(Flatten())

model_3.add(Dense(64, activation='relu'))
model_3.add(Dense(64, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))

model_3.add(Dense(8, activation='softmax'))



In [ ]:
model_3.summary()

In [ ]:
model_3.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model_3.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=20, class_weight=class_weights, callbacks=callbacks)


In [ ]:
# Loss Curve

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy Curve

plt.plot(history.history['accuracy'], label='Train accuracy')
plt.plot(history.history['val_accuracy'], label='Val accuracy')
plt.xlabel('Ephocs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
score, acc = model_3.evaluate(x=test_batches, steps=len(test_batches))

print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
predictions = model_3.predict(x=test_batches, steps=len(test_batches))
predictions

In [ ]:
predictions = np.argmax(predictions, axis=-1)
predictions

In [ ]:
print("Classification Report: \n", classification_report(test_batches.classes, predictions))

In [ ]:
#Saving the model

fer_json = model_3.to_json()
with open("path\\model_3.json", "w") as json_file:
    json_file.write(fer_json)
model_3.save_weights("path\\Saved Model\\model_3.h5")

# Custom Model 4

In [ ]:
model_4 = Sequential()

model_4.add(Conv2D(8, (3, 3), padding='same', activation = 'relu', input_shape=(234, 234, 3)))
model_4.add(Conv2D(16, (3, 3), padding='same', activation = 'relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model_4.add(Dropout(0.5))

model_4.add(Conv2D(32, (5, 5), padding='same', activation = 'relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model_4.add(Dropout(0.5))

model_4.add(Conv2D(64, (5, 5), padding='same', activation = 'relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model_4.add(Dropout(0.5))

model_4.add(Conv2D(64, (5, 5), padding='same', activation = 'relu'))
model_4.add(BatchNormalization())
model_4.add(MaxPool2D(pool_size=(4, 4), padding='same'))
model_4.add(Dropout(0.5))

model_4.add(Flatten())

model_4.add(Dense(64, activation='relu'))
model_4.add(Dense(64, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))

model_4.add(LeakyReLU(alpha=0.1))
model_4.add(Dense(8, activation='softmax'))



In [ ]:
model_4.summary()

In [ ]:
model_4.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model_4.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=100, class_weight=class_weights, callbacks=callbacks)


In [ ]:
# Loss Curve

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy Curve

plt.plot(history.history['accuracy'], label='Train accuracy')
plt.plot(history.history['val_accuracy'], label='Val accuracy')
plt.xlabel('Ephocs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
score, acc = model_4.evaluate(x=test_batches, steps=len(test_batches))

print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
predictions = model_4.predict(x=test_batches, steps=len(test_batches))
predictions

In [ ]:
predictions = np.argmax(predictions, axis=-1)
predictions

In [ ]:
print("Classification Report: \n", classification_report(test_batches.classes, predictions))

In [ ]:
#Saving the model

fer_json = model_4.to_json()
with open("path\\model_4.json", "w") as json_file:
    json_file.write(fer_json)
model_4.save_weights("path\\model_4.h5")

# Custom Model 5 (from paper)

In [ ]:
model_5 = Sequential()

model_5.add(Conv2D(8, (5, 11), strides=1, padding='same', activation = 'relu', input_shape=(234, 234, 3)))
model_5.add(Conv2D(16, (5, 11), strides=1, padding='same', activation = 'relu'))
model_5.add(MaxPool2D(pool_size=(2, 2), strides=2))

model_5.add(Flatten())

model_5.add(Dense(144, activation='relu'))
model_5.add(Dense(8, activation='softmax'))


In [ ]:
model_5.summary()

In [ ]:
model_5.compile(optimizer = 'adadelta', loss = 'mean_squared_error', metrics = ['accuracy'])

In [ ]:
history = model_5.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=50, class_weight=class_weights, callbacks=callbacks)


In [ ]:
# Loss Curve

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy Curve

plt.plot(history.history['accuracy'], label='Train accuracy')
plt.plot(history.history['val_accuracy'], label='Val accuracy')
plt.xlabel('Ephocs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
score, acc = model_5.evaluate(x=test_batches, steps=len(test_batches))

print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
predictions = model_5.predict(x=test_batches, steps=len(test_batches))
predictions

In [ ]:
predictions = np.argmax(predictions, axis=-1)
predictions

In [ ]:
print("Classification Report: \n", classification_report(test_batches.classes, predictions))

In [ ]:
#Saving the model

fer_json = model_5.to_json()
with open("path\\model_5.json", "w") as json_file:
    json_file.write(fer_json)
model_5.save_weights("path\\model_5.h5")

# MESO4

In [ ]:
model_1 = Sequential()
model_1.add(Conv2D(8, (3, 3), padding='same', activation = 'relu', input_shape=(234, 234, 3)))
model_1.add(BatchNormalization())
model_1.add(Conv2D(64, (3, 3), activation='relu'))
model_1.add(MaxPool2D(pool_size=(2, 2), padding='same'))


model_1.add(Conv2D(8, (5, 5), padding='same', activation = 'relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPool2D(pool_size=(2, 2), padding='same'))

model_1.add(Conv2D(16, (5, 5), padding='same', activation = 'relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPool2D(pool_size=(2, 2), padding='same'))

model_1.add(Conv2D(16, (5, 5), padding='same', activation = 'relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPool2D(pool_size=(4, 4), padding='same'))

model_1.add(Flatten())

model_1.add(Dropout(0.5))

model_1.add(Dense(16))
model_1.add(LeakyReLU(alpha=0.1))

model_1.add(Dropout(0.5))
model_1.add(Dense(8, activation='softmax'))

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model_1.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=50, class_weight=class_weights, callbacks=callbacks)


In [ ]:
# Loss Curve

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy Curve

plt.plot(history.history['accuracy'], label='Train accuracy')
plt.plot(history.history['val_accuracy'], label='Val accuracy')
plt.xlabel('Ephocs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
score, acc = model_1.evaluate(x=test_batches, steps=len(test_batches))

print('Test score:', score)
print('Test accuracy:', acc)

# Pre-Trained Models

# VGG 16

In [ ]:
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(234,234,3))

In [ ]:
base_model_vgg16.summary()

In [ ]:
for layer in base_model_vgg16.layers:
    base_model_vgg16.trainable = True

In [ ]:
for i, layer in enumerate(base_model_vgg16.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
model_vgg16 = Sequential()
model_vgg16.add(base_model_vgg16)
model_vgg16.add(Flatten())
model_vgg16.add(Dense(144, activation = 'relu'))
model_vgg16.add(Dense(9, activation='softmax'))

In [ ]:
model_vgg16.summary()

In [ ]:
model_vgg16.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy', f1])

In [ ]:
history = model_vgg16.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=20, class_weight=class_weights, callbacks=callbacks)

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
score, acc, f1 = model_vgg16.evaluate(x=test_batches, steps=len(test_batches))

print('Test score:', score)
print('Test accuracy:', acc)
print('F1 score: ', f1)

In [ ]:
#Saving the model

fer_json = model_vgg16.to_json()
with open("path\\model_vgg16.json", "w") as json_file:
    json_file.write(fer_json)
model_vgg16.save_weights("path\\model_vgg16.h5")

## VGG 19

In [ ]:
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(234,234,3))

In [ ]:
base_model_vgg19.summary()

In [ ]:
for layer in base_model_vgg19.layers:
    base_model_vgg19.trainable = True

In [ ]:
for i, layer in enumerate(base_model_vgg19.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
model_vgg19 = Sequential()
model_vgg19.add(base_model_vgg19)
model_vgg19.add(Flatten())
model_vgg19.add(Dense(8, activation='softmax'))

In [ ]:
model_vgg19.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model_vgg19.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=20, class_weight=class_weights)

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
score, acc = model_vgg19.evaluate(x=test_batches, steps=len(test_batches))

print('Test score:', score)
print('Test accuracy:', acc)

# INCEPTION V3

In [ ]:
base_model_inceptionv3 = InceptionV3(weights=None, include_top=False, input_shape=(234,234,3))

In [ ]:
base_model_inceptionv3.summary()

In [ ]:
for layer in base_model_inceptionv3.layers:
    base_model_inceptionv3.trainable = True

In [ ]:
for i, layer in enumerate(base_model_inceptionv3.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
model_inceptionv3 = Sequential()
model_inceptionv3.add(base_model_inceptionv3)
model_inceptionv3.add(Flatten())
model_inceptionv3.add(Dense(64, activation = 'relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),activity_regularizer=regularizers.l2(1e-5)))
model_inceptionv3.add(Dense(9, activation='softmax'))

In [ ]:
model_inceptionv3.summary()

In [ ]:
model_inceptionv3.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model_inceptionv3.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=25, class_weight=class_weights)

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
score, acc = model_inceptionv3.evaluate(x=test_batches, steps=len(test_batches))

print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
#Saving the model

fer_json = model_inceptionv3.to_json()
with open("path\\model_inceptionv3.json", "w") as json_file:
    json_file.write(fer_json)
model_inceptionv3.save_weights("path\\model_inceptionv3.h5")

## EfficientNet B0

In [ ]:
base_model_efficientnet = efn.EfficientNetB0(weights='imagenet',include_top=False,pooling='max',input_shape=(234, 234, 3))
base_model_efficientnet.summary()

In [ ]:
for layer in base_model_efficientnet.layers:
    base_model_efficientnet.trainable = True
    
for i, layer in enumerate(base_model_efficientnet.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
model_efficientnet = Sequential()
model_efficientnet.add(base_model_efficientnet)
model_efficientnet.add(Flatten())
model_efficientnet.add(Dense(8, activation='softmax'))

In [ ]:
model_efficientnet.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model_efficientnet.fit(x=train_batches, steps_per_epoch=len(train_batches), validation_data=validation_batches,
           validation_steps=len(validation_batches), epochs=25, class_weight=class_weights)

In [ ]:
# Loss Curve

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Ephocs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy Curve

plt.plot(history.history['accuracy'], label='Train accuracy')
plt.plot(history.history['val_accuracy'], label='Val accuracy')
plt.xlabel('Ephocs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
score, acc = model_efficientnet.evaluate(x=test_batches, steps=len(test_batches))

print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
#Saving the model

fer_json = model_efficientnet.to_json()
with open("path\\model_efficientnet.json", "w") as json_file:
    json_file.write(fer_json)
model_efficientnet.save_weights("path\\model_efficientnet.h5")

# Confution Matrix

In [ ]:
# cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#             horizontalalignment="center",
#             color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

In [ ]:
# test_batches.class_indices

In [ ]:
# cm_plot_labels = ['PB','L']
# plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

# Prediction

In [ ]:
test_img = glob.glob('Image _for_Prediction/*.png')
test_img

In [ ]:
def make_perdiction_on_bb(img):
    test_image = image.load_img(img, target_size =(234,234))
    plt.imshow(test_image)
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image) 
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    beat = ['LBB', 'FPNB']
    result_in_number = [result[0][0],result[0][1]]
    ax.bar(beat,result_in_number)
    plt.show()

In [ ]:
for img in test_img:
    make_perdiction_on_bb(img)

#### Single Image

In [ ]:
# test_img = os.path.join(test_path, 'FPNB', '4.png')
# test_img

In [ ]:
# test_image = image.load_img(test_img, target_size =(234,234))
# test_image

In [ ]:
# test_image = image.load_img(test_img, target_size =(234,234))
# #test_image = image.load_img('path', target_size =(128,192))
# plt.imshow(test_image)
# test_image = image.img_to_array(test_image)
# test_image = np.expand_dims(test_image, axis = 0)
# result = model.predict(test_image) 
# result

In [ ]:
# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# beat = ['LBB', 'FPNB']
# result_in_number = [result[0][0],result[0][1]]
# ax.bar(beat,result_in_number)
# plt.show()

In [ ]:
# predictions = model.predict(x=test_batches, steps=len(test_batches))

In [ ]:
# np.round(predictions)

In [ ]:
# model = model_from_json(open("path\\model_234x234.json", "r").read())
# model.load_weights('path\\model_234x234.h5')